In [8]:
import glob
import shutil
import pandas as pd
import re
import numpy as np
import os

# read csv with coll codes for unwanted cifs

In [9]:
csv_paths = glob.glob('icsd_theoretical_and_hybrid/*')
csv_dfs = [pd.read_csv(csv_path,delimiter='\t') for csv_path in csv_paths]

coll_codes_to_remove = pd.concat(csv_dfs)['CollectionCode'].to_numpy()

# gather coll codes for all downloaded cifs

In [25]:
downloaded_cifs_dir = 'unzipped_downloaded_cifs'
downloaded_cif_paths = np.array(glob.glob(f'{downloaded_cifs_dir}/*'))
downloaded_cif_coll_codes = [int(re.search('(\S*)_CollCode(\S*).cif',path).groups()[1]) for path in downloaded_cif_paths]

# subtract out unwanted coll codes

In [26]:
coll_codes_to_keep = list(set(downloaded_cif_coll_codes) - set(coll_codes_to_remove))

print(f'Started with {len(downloaded_cif_coll_codes)}.')
print(f'Removed {len(coll_codes_to_remove)}.')
print(f'Remaining {len(coll_codes_to_keep)}.')

Started with 17171.
Removed 2497.
Remaining 14679.


# move the cif files we want to keep to a new directory

In [27]:
filtered_cif_dir = 'final_cifs_for_aflow'

if os.path.isdir(filtered_cif_dir): shutil.rmtree(filtered_cif_dir)
os.mkdir(filtered_cif_dir)


# determine paths of cif files to keep
keep_cif_path_bool = np.isin(downloaded_cif_coll_codes,coll_codes_to_keep)
paths_to_keep = downloaded_cif_paths[keep_cif_path_bool]

# move cif files

for path in paths_to_keep:
    destination_path = path.replace(downloaded_cifs_dir,filtered_cif_dir)
    shutil.move(path,destination_path)